# Using ESCO to identify positions associated with a given skill.

ESCO contains a matching between skills and occupations.
This notebook shows how to use this matching to identify occupations.

In [ ]:
# Get all occupations and related skills.
import pandas as pd
import io

import esco

db = esco.LocalDB()
skills = db.load_skills()
occupations = db.load_occupations()


In [ ]:
# Get the row from skills that matches the index value 'x'Arith
db.get_label('http://data.europa.eu/esco/skill/000f1d3d-220f-4789-9c0a-cc742521fb02')


In [ ]:
with pd.option_context("max_colwidth", None):
  display(
    occupations[occupations.description.str.contains("architect", case=False)][['label', 'skill_', 'knowledge_', 'description']]
    )

In [ ]:
my_skills = {
        "http://data.europa.eu/esco/skill/19a8293b-8e95-4de3-983f-77484079c389":
          { "label": "Java (computer programming)" },
        "http://data.europa.eu/esco/skill/21d2f96d-35f7-4e3f-9745-c533d2dd6e97":
          { "label": "computer programming" },
        "http://data.europa.eu/esco/skill/43ae58b9-5e56-4524-b45a-b422777a0576":
          { "label": "database" },
        "http://data.europa.eu/esco/skill/4da171e5-779c-4983-a76f-91c16751e99f":
          { "label": "MySQL" },
        "http://data.europa.eu/esco/skill/598de5b0-5b58-4ea7-8058-a4bc4d18c742":
          { "label": "SQL" },
        "http://data.europa.eu/esco/skill/6f8a40d6-f9ce-43ec-a72f-d4213a53f3ed":
          { "label": "Ansible" },
        "http://data.europa.eu/esco/skill/76ef6ed3-1658-4a1a-9593-204d799c6d0c":
          { "label": "NoSQL" },
        "http://data.europa.eu/esco/skill/9983816d-cc78-4d3f-9e3c-c7baa9ebc77a":
          { "label": "computer equipment" },
        "http://data.europa.eu/esco/skill/a57a54b6-2f2e-43e4-9621-b52f4a63cb08":
          { "label": "LDAP" },
        "http://data.europa.eu/esco/skill/ab1e97ed-2319-4293-a8b7-072d2648822f":
          { "label": "database management systems" },
        "http://data.europa.eu/esco/skill/b16bcbcb-1d3f-42b3-a6a5-b91348a72b70":
          { "label": "Jboss" },
        "http://data.europa.eu/esco/skill/ccd0a1d9-afda-43d9-b901-96344886e14d":
          { "label": "Python (computer programming)" },
      }
my_skills_keys= set(my_skills.keys())
my_skills_keys

In [ ]:
# Create new dataframe containing: all the o_index columns, plus a new column containing all the matching skills for each occupation.
occupations_for_my_skills = occupations.apply(
    lambda o: {
        "label": o.label,
        "skills": o.skill,
        "matching": [
            skills[skills.index == skill_uri].label.values[0]
            for skill_uri in (set(o.s) & my_skills_keys)
        ],
    },
    axis=True,
    result_type="expand",
)
occupations_for_my_skills[occupations_for_my_skills.matching.apply(lambda x: len(x) > 0)]

In [ ]:
skill_python = db.search_products({'python'})
skill_python[0]['uri']

In [ ]:
import esco.sparql

skill_set = ', '.join(['<'+s+'>' for s in list(my_skills)[1:2]])
print(skill_set)
sparql = esco.sparql.SparqlClient()
occupations_with_python = sparql.query(
"""
SELECT DISTINCT * 
WHERE {
  ?o a esco:Occupation ;
    esco:relatedEssentialSkill """ + skill_set + """ ;
    skos:prefLabel ?occupation 
    .

    FILTER (lang(?occupation) = "en")
}
""")
occupations_with_python = pd.read_csv(io.StringIO(occupations_with_python.decode('utf-8')))
occupations_with_python.head()

In [ ]:
C = pd.DataFrame(data= [(0, 1, 2), (3, 4, 5)], columns=['a', 'b', 'c'])
C

In [ ]:
C.apply(lambda x: {'Z': x.a}, axis=True,result_type='expand')[lambda x: x.Z > 1]
# Show only the rows where Z > 1